# Import libraries

In [1]:
!pip install sklearn
!pip install pandas
!pip install matplotlib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# empty list to append metric values
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []

# parameters
steps = 96
n_hidden = 1
units = 150
batch_size = 96
epochs = 100

# months to evaluate model on
date = 2018090000

# for later use
features_num = 14

In [2]:
data = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values in the whole data set
data.fillna(data.mean(), inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Required functions to put data into required shape for LSTM:

In [3]:
# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# Set up X and y for train, test and val into correct shapes

In [4]:
# divide features and labels
X_train = data_train[:, 0:14] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:14]
y_test = data_test[:, -1] 

# divide data into validation and normal test 
X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

# cut data
X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# Import libraries:

In [5]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

# Design LSTM:

In [6]:
def regressor_tunning(kernel_initializer = 'he_uniform',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 1:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    optimizer = optimizers.RMSprop()
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
    return model

model = regressor_tunning()

# Train the model:

In [7]:
history = model.fit(X_train,
                    y_train, 
                    batch_size = batch_size, 
                    epochs = epochs,
                    shuffle = False, 
                    validation_data = (X_val, y_val))
    
# required before predicitons
model.reset_states()
    
y_pred = model.predict(X_test, batch_size = batch_size)

Epoch 1/100
43/43 [==============================] - 32s 750ms/step - loss: 0.3126 - mse: 0.3126 - mae: 0.3781 - val_loss: 0.0182 - val_mse: 0.0182 - val_mae: 0.1333
Epoch 2/100
43/43 [==============================] - 28s 656ms/step - loss: 0.1080 - mse: 0.1080 - mae: 0.2620 - val_loss: 0.0101 - val_mse: 0.0101 - val_mae: 0.0979
Epoch 3/100
43/43 [==============================] - 26s 613ms/step - loss: 0.0587 - mse: 0.0587 - mae: 0.1914 - val_loss: 5.3386e-04 - val_mse: 5.3386e-04 - val_mae: 0.0134
Epoch 4/100
43/43 [==============================] - 27s 618ms/step - loss: 0.0318 - mse: 0.0318 - mae: 0.1418 - val_loss: 6.2931e-04 - val_mse: 6.2931e-04 - val_mae: 0.0219
Epoch 5/100
43/43 [==============================] - 26s 601ms/step - loss: 0.0185 - mse: 0.0185 - mae: 0.1078 - val_loss: 0.0016 - val_mse: 0.0016 - val_mae: 0.0345
Epoch 6/100
43/43 [==============================] - 26s 606ms/step - loss: 0.0093 - mse: 0.0093 - mae: 0.0746 - val_loss: 0.0033 - val_mse: 0.0033 - val_

Epoch 48/100
43/43 [==============================] - 29s 677ms/step - loss: 9.7664e-04 - mse: 9.7664e-04 - mae: 0.0150 - val_loss: 3.1054e-04 - val_mse: 3.1054e-04 - val_mae: 0.0099
Epoch 49/100
43/43 [==============================] - 28s 652ms/step - loss: 9.6723e-04 - mse: 9.6723e-04 - mae: 0.0148 - val_loss: 2.9835e-04 - val_mse: 2.9835e-04 - val_mae: 0.0104
Epoch 50/100
43/43 [==============================] - 28s 660ms/step - loss: 9.5443e-04 - mse: 9.5443e-04 - mae: 0.0147 - val_loss: 2.9671e-04 - val_mse: 2.9671e-04 - val_mae: 0.0111
Epoch 51/100
43/43 [==============================] - 28s 652ms/step - loss: 9.4457e-04 - mse: 9.4457e-04 - mae: 0.0145 - val_loss: 3.0626e-04 - val_mse: 3.0626e-04 - val_mae: 0.0103
Epoch 52/100
43/43 [==============================] - 28s 658ms/step - loss: 9.5139e-04 - mse: 9.5139e-04 - mae: 0.0147 - val_loss: 2.8798e-04 - val_mse: 2.8798e-04 - val_mae: 0.0098
Epoch 53/100
43/43 [==============================] - 28s 648ms/step - loss: 9.2179e-

Epoch 93/100
43/43 [==============================] - 16s 363ms/step - loss: 8.9820e-04 - mse: 8.9820e-04 - mae: 0.0140 - val_loss: 2.7447e-04 - val_mse: 2.7447e-04 - val_mae: 0.0102
Epoch 94/100
43/43 [==============================] - 16s 371ms/step - loss: 8.2752e-04 - mse: 8.2752e-04 - mae: 0.0132 - val_loss: 2.8839e-04 - val_mse: 2.8839e-04 - val_mae: 0.0097
Epoch 95/100
43/43 [==============================] - 17s 387ms/step - loss: 8.6802e-04 - mse: 8.6802e-04 - mae: 0.0138 - val_loss: 2.8375e-04 - val_mse: 2.8375e-04 - val_mae: 0.0106
Epoch 96/100
43/43 [==============================] - 17s 404ms/step - loss: 9.1804e-04 - mse: 9.1804e-04 - mae: 0.0139 - val_loss: 2.8741e-04 - val_mse: 2.8741e-04 - val_mae: 0.0107
Epoch 97/100
43/43 [==============================] - 16s 377ms/step - loss: 8.5496e-04 - mse: 8.5496e-04 - mae: 0.0135 - val_loss: 3.7830e-04 - val_mse: 3.7830e-04 - val_mae: 0.0108
Epoch 98/100
43/43 [==============================] - 17s 395ms/step - loss: 8.8089e-

# Evaluate predictions:

In [8]:
# cannot use inverse function; prices col = 14
y_pred = (y_pred * sc_X.data_range_[14]) + (sc_X.data_min_[14])
y_test = (y_test * sc_X.data_range_[14]) + (sc_X.data_min_[14])

# Reshaping
y_pred = np.reshape(y_pred, (y_pred.shape[0]))

# =============================================================================
# METRICS EVALUATION (1) for the whole test set
# =============================================================================
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

# calculate metrics
rmse_error = mse(y_test, y_pred, squared = False)
mae_error = mae(y_test, y_pred)

# append to list
rmse_gen.append(rmse_error)
mae_gen.append(mae_error)

# Need to process data with spike occurences in the same way as features and offers:
(Required to evaluate predictions in both normal regions and spike regions)

In [9]:
# download data for shaded area
data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# make sure shaded area will correspond to values outputed by LSTM
data.reset_index(drop = True, inplace = True)

# fill_nan is already made - so lets split data into test and train
from sklearn.model_selection import train_test_split

# divide data into train and test 
shade_train, shade_test = train_test_split(
         data, test_size = 0.15, shuffle = False)

# reset index of testing data
shade_test.reset_index(drop = True, inplace = True)

# function to split data into correct shape for RNN
def split_data(shade_test, steps):
    y_spike_occ = list()
    upper_lim = list()
    lower_lim = list()
    for i in range(steps, len(shade_test.index)):
        y_spike_occ.append(shade_test['spike_occurance'][i])
        upper_lim.append(shade_test['spike_upperlim'][i])
        lower_lim.append(shade_test['spike_lowerlim'][i])
    return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

y_spike_occ, spike_upperlim, spike_lowerlim = split_data(shade_test, steps)
y_spike_occ = cut_data(y_spike_occ, batch_size)
spike_upperlim = cut_data(spike_upperlim, batch_size)
spike_lowerlim = cut_data(spike_lowerlim, batch_size)

# Evaluate on spike and normal regions

In [16]:
# =============================================================================
# METRICS EVALUATION (2) on spike regions
# =============================================================================

# smal adjustment
#y_test.replace(0, 0.0001,inplace = True)

# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mae_spike = mae(y_test_spike, y_pred_spike)

# append ot lists
rmse_spi.append(rmse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# METRIC EVALUATION (3) on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mae_normal = mae(y_test_normal, y_pred_normal)

# append to list
rmse_nor.append(rmse_normal)
mae_nor.append(mae_normal)

# Now evaluate predictions on Spike & Normal regions:

In [39]:
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

# Save y_pred & spike limits to plot with Stateless LSTM
datasave = pd.DataFrame({'y_pred': y_pred,
                         'y_test':y_test,
                         'spike_upperlim':spike_upperlim,
                         'spike_lowerlim':spike_lowerlim})

datasave.to_csv('LSTM_plot_Stateful_4m.csv')

In [40]:
results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,25.538148,13.986985,61.526372,48.157042,12.918564,8.69448
